In [1]:
from sklearn.feature_extraction.image import extract_patches_2d
from sklearn.feature_extraction import image
import numpy as np
import cv2


/home/abhishek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/abhishek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/abhishek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/abhishek/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
img = cv2.imread('2.jpg')

#img = cv2.resize((317,425))

#img = cv2.resize(img,(80,80))
m,n,l = img.shape

b,g,r = cv2.split(img)

window_shape = (6, 6)

In [3]:
blue = extract_patches_2d(b, window_shape)
green = extract_patches_2d(g, window_shape)
red = extract_patches_2d(r, window_shape)


/home/abhishek/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/image.py:287: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  indexing_strides = arr[slices].strides


In [4]:
Dg = []
Db = []

Bl_pixels = []
Gr_pixels = []
Re_pixels = []


In [5]:
for i in range(len(blue)):
    bl = blue[i]
    gr = green[i]
    re = red[i]
    
    bls = list(bl.flat)
    grls = list(gr.flat)
    rels = list(re.flat)
    
    if np.var(bl) < 0.9 or np.var(gr) < 0.9 or np.var(re) < 0.9:
        Db.extend([a - b for a, b in zip(bls, rels)])
        Dg.extend([a - b for a, b in zip(grls, rels)])

        Bl_pixels.extend(bls)
        Gr_pixels.extend(grls)
        Re_pixels.extend(rels)

In [6]:
maxB = Db[0]
maxG = Dg[0]
indB = 0
indG = 0

'''for i in range(1,len(Db)):
    if maxB < Db[i]:
        maxB = Db[i]
        indB = i
    if maxG < Dg[i]:
        maxG = Dg[i]
        indG = i
'''
ind = 0
maxR = 0
for i in range(1,len(Db)):
    if maxB < Db[i] or maxG < Dg[i]:
        if maxR < Re_pixels[i]:
            ind = i
            maxB = Db[i]
            maxG = Dg[i]
            maxR = Re_pixels[i]
'''
if maxB > maxG :
    indG = indB
else:
    indB = indG
'''

'\nif maxB > maxG :\n    indG = indB\nelse:\n    indB = indG\n'

In [7]:
#Backlight Pixels
Backlight = []

Backlight.append(Bl_pixels[ind])
Backlight.append(Gr_pixels[ind])
Backlight.append(Re_pixels[ind])


In [8]:
Backlight

[150, 176, 86]

In [9]:
reconsBlue = image.reconstruct_from_patches_2d(blue, (m,n))
reconsGreen = image.reconstruct_from_patches_2d(green, (m,n))
reconsRed = image.reconstruct_from_patches_2d(red, (m,n))


In [10]:
final_img = np.zeros((img.shape[0], img.shape[1], 3))

final_img[:,:,0] = reconsBlue
final_img[:,:,1] = reconsGreen
final_img[:,:,2] = reconsRed


In [11]:
cv2.imwrite('FinalImpl1.jpg',final_img)


True

In [12]:
i_ind = 0
j_ind = 0
flag = 0
for i in range(m):
    for j in range(n):
        if Backlight == list(img[i,j]):
                i_ind = i
                j_ind = j
                img[i,j] = 0
                img[i+1,j] = 0
                img[i,j+1]=0
                img[i-1,j]=0
                img[i-1,j-1]=0
                img[i+1,j+1]=0
                #flag = 1
                #break   
        #if flag == 1:
            #break

In [13]:
cv2.imwrite('FinalImpl-JN2.jpg',img)

True

In [14]:
#Finding attenuation ratios 

Sb = (-0.00113*450 + 1.6251)
Sg = (-0.00113*540 + 1.6251)
Sr = (-0.00113*620 + 1.6251)

#Attenuation ratios

lamda_b = (Sb*Backlight[2])/(Sr*Backlight[0])

lamda_g = (Sg*Backlight[2])/(Sr*Backlight[1])

print('lamda_b : ',lamda_b, ' lamda_g : ',lamda_g)

lamda_b :  0.6924651162790698  lamda_g :  0.536416490486258


In [15]:
#Finding Initial Transmission of Red Channel

tr = []
for i in range(m):
    trans = []
    for j in range(n):
        trans.append((r[i][j] - Backlight[2])/np.max([(r[i][j] - Backlight[k]) for k in range(len(Backlight))]))
    tr.append(trans)

print(tr)

/home/abhishek/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in ubyte_scalars
  import sys
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
#Refining Transmission using WLS Filter

r_trans = np.array(tr)
print(r_trans)

L = np.log(r_trans + np.spacing(1))

alpha = 1.2

lamda = 0.05

smallNum = 0.0001

r,c = r_trans.shape

k = r*c

dy = np.diff(L,0,0)

old_dy = dy
absDy = abs(dy)

power = (np.power(absDy,alpha) + smallNum)

dy = np.divide(lamda,power)

dy = np.pad(dy,(1,0),'constant')

dy = dy.flatten()

[[1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 ...
 [0.         1.         1.         ... 0.3140496  0.32244897 0.32520324]
 [1.         0.01030928 1.         ... 0.30833334 0.33333334 0.32244897]
 [1.         0.01030928 1.         ... 0.31120333 0.31967214 0.30833334]]


In [17]:
dx = np.diff(L,0,1)

old_dx = dx

absDx = abs(dx)

powerDx = (np.power(absDx,alpha) + smallNum)

dx = np.divide(lamda,powerDx)

dx = np.pad(dx,(0,1),'constant')

dx = dx.flatten()

In [18]:
lapMat = np.zeros(shape=(len(dx),2))

lapMat[:,0] = dx
lapMat[:,1] = dy

d = [-r,-1]

lapMat = np.transpose(lapMat)

import scipy

reslapMat = scipy.sparse.spdiags(lapMat,d,k,k)


In [19]:
reslapMat.shape

(538900, 538900)

In [20]:
e = dx

w = np.lib.pad(dx,(r,0),'constant',constant_values=(0))


In [21]:
w = w[0:len(w)-r]

In [22]:
s = dy

n = np.lib.pad(dy,(1,0),'constant',constant_values=(0))

n = n[0:(len(n) - 1)]


In [23]:
e.shape,w.shape,s.shape,n.shape,dy.shape

((540385,), (540385,), (540385,), (540385,), (540385,))

In [24]:
D = 1-(e+w+s+n);

In [65]:
A = reslapMat

temp = np.transpose(D)

A = A + np.transpose(A) + scipy.sparse.spdiags(temp, 0, k, k);



In [66]:
A

<538900x538900 sparse matrix of type '<class 'numpy.float64'>'
	with 2688998 stored elements in Compressed Sparse Row format>

In [67]:
A.shape

(538900, 538900)

In [31]:
type(A)

scipy.sparse.csr.csr_matrix

In [43]:
tr_flat = r_trans.flatten()

import numpy.linalg as lin

x_out,resid_out,rank_out,s_out = np.linalg.lstsq(A,tr_flat)



LinAlgError: 0-dimensional array given. Array must be two-dimensional

In [57]:
from scipy.sparse.linalg import lsqr

#This part of code is taking lot of time to run. Please check this.

x_out,resid_out,rank_out,s_out = lsqr(A,tr_flat)[:4]


KeyboardInterrupt: 

In [70]:


#x = np.linalg.solve(A, np.transpose(tr_flat))


from scipy import linalg
x = linalg.solve(A, tr_flat)

ValueError: Sparse matrices are not supported by this function. Perhaps one of the scipy.sparse.linalg functions would work instead.